***WASTE SORTING***

Image classification

Incorrect waste disposal can create a number of problems for both public administrations (waste of time and resources due to reprocessing) and the environment (contamination). These problems can be avoided with automatic waste sorting. 
For this reason, this image classifier has been created, in order to know how to classify waste and materials.

In [1]:

import pandas as pd
import numpy as np
import os
import zipfile as zf
import shutil
import re
import seaborn as sns
from pathlib import Path
from random import choice, randrange

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib.image import imread
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dropout,Flatten,Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential,load_model
from sklearn import model_selection
from os import listdir
from os.path import isfile, join
import tensorflow as tf
from skimage.io import imread
import cv2
import re
import glob
from sklearn import preprocessing

C:\Users\Mary\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [3]:
dir = os.path.dirname
src_path = dir(os.path.abspath(''))

X TRAIN

*Descargar datos para X_train e y_train* *texto en cursiva*

In [4]:
path_train = src_path + os.sep + "data" +  os.sep + "train"  +  os.sep + '**'  +  os.sep + '**.jpg'
path_train

'c:\\Users\\Mary\\Desktop\\BootCamp\\Python\\MaryC-MezaR\\data_science_bootcamp_2021\\Machine_Learning_project\\data\\train\\**\\**.jpg'

In [5]:
path = Path(os.getcwd())/"data"
path

WindowsPath('c:/Users/Mary/Desktop/BootCamp/Python/MaryC-MezaR/data_science_bootcamp_2021/Machine_Learning_project/notebooks/data')

In [6]:

def make_imag_df(path):
    dictionary = {}
    for paths in glob.glob(path):
        dictionary.setdefault('conjunto', []).append(paths.split('\\')[-3])
        dictionary.setdefault('name', []).append(paths.split('\\')[-2])
        dictionary.setdefault('id', []).append(paths.split('\\')[-1][:-4])
        dictionary.setdefault('img', []).append(paths.split('\\')[-1])
        
    images = pd.DataFrame(dictionary)
    return images

train_set = make_imag_df(path_train)

In [7]:
train_set

,conjunto,name,id,img
0,train,battery,battery1,battery1.jpg
1,train,battery,battery10,battery10.jpg
2,train,battery,battery100,battery100.jpg
3,train,battery,battery102,battery102.jpg
4,train,battery,battery103,battery103.jpg
...,...,...,...,...
4798,train,plastic,plastic92,plastic92.jpg
4799,train,plastic,plastic94,plastic94.jpg
4800,train,plastic,plastic96,plastic96.jpg
4801,train,plastic,plastic97,plastic97.jpg


In [7]:
#funcion columna
def creat_colum (dataframe):
    dataframe['path']= dataframe.name + "/" + dataframe.img
    dataframe['label'] = dataframe['name'].map({'battery': 0, 'biological': 1, 'cardboard': 2, 'glass':3, 'metal':4, 'paper':5, 'plastic':6})

    return dataframe

In [8]:
train_set = creat_colum (train_set)

In [4]:
train_set

NameError: name 'train_set' is not defined

In [9]:
# path para importar las imagenes 
path_path = src_path + os.sep + "data" + os.sep + "train"  +  os.sep 
path_path

'c:\\Users\\Mary\\Desktop\\BootCamp\\Python\\MaryC-MezaR\\data_science_bootcamp_2021\\Machine_Learning_project\\data\\train\\'

In [13]:
imagenes = []
size = []
#width = []

for i in train_set.path:
    x = path_path + i
    imagenes_rs = cv2.imread(x, 1)
    imagenes.append(cv2.resize(imagenes_rs, (224, 224)))
    size.append(imagenes_rs.shape)
    #width.append(np.size(imagenes_rs, 1))
    
train_set["nombre_img"] = imagenes
train_set['size'] = '(224,224)'
#train_set["size"] = size

In [28]:
# nd =train_set.assign(size=(224,224))

train_set['size'] = '(224,224)'

In [29]:
train_set

,conjunto,name,id,img,path,label,nombre_img,size
0,train,battery,battery1,battery1.jpg,battery/battery1.jpg,0,"[[[189, 199, 206], [189, 199, 206], [189, 199,...","(224,224)"
1,train,battery,battery10,battery10.jpg,battery/battery10.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","(224,224)"
2,train,battery,battery100,battery100.jpg,battery/battery100.jpg,0,"[[[151, 155, 165], [148, 153, 162], [152, 157,...","(224,224)"
3,train,battery,battery102,battery102.jpg,battery/battery102.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","(224,224)"
4,train,battery,battery103,battery103.jpg,battery/battery103.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","(224,224)"
...,...,...,...,...,...,...,...,...
4798,train,plastic,plastic92,plastic92.jpg,plastic/plastic92.jpg,6,"[[[176, 166, 166], [183, 173, 173], [171, 163,...","(224,224)"
4799,train,plastic,plastic94,plastic94.jpg,plastic/plastic94.jpg,6,"[[[217, 203, 204], [216, 202, 203], [215, 201,...","(224,224)"
4800,train,plastic,plastic96,plastic96.jpg,plastic/plastic96.jpg,6,"[[[237, 224, 222], [234, 221, 219], [230, 217,...","(224,224)"
4801,train,plastic,plastic97,plastic97.jpg,plastic/plastic97.jpg,6,"[[[94, 123, 154], [94, 123, 154], [94, 123, 15...","(224,224)"


In [19]:
from PIL import Image
size = []

for i in train_set.path:
    x = Image.open(i)
    size.append(im.size)
    #x = path_path + i
    #imagenes_rs = cv2.imread(x, 1)
    #size.append(imagenes_rs.shape)

train_set["size"] = size

FileNotFoundError: [Errno 2] No such file or directory: 'battery/battery1.jpg'

In [17]:
train_set

,conjunto,name,id,img,path,label,nombre_img,size
0,train,battery,battery1,battery1.jpg,battery/battery1.jpg,0,"[[[189, 199, 206], [189, 199, 206], [189, 199,...","(180, 280, 3)"
1,train,battery,battery10,battery10.jpg,battery/battery10.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","(165, 220, 3)"
2,train,battery,battery100,battery100.jpg,battery/battery100.jpg,0,"[[[151, 155, 165], [148, 153, 162], [152, 157,...","(183, 275, 3)"
3,train,battery,battery102,battery102.jpg,battery/battery102.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","(224, 224, 3)"
4,train,battery,battery103,battery103.jpg,battery/battery103.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","(188, 268, 3)"
...,...,...,...,...,...,...,...,...
4798,train,plastic,plastic92,plastic92.jpg,plastic/plastic92.jpg,6,"[[[176, 166, 166], [183, 173, 173], [171, 163,...","(384, 512, 3)"
4799,train,plastic,plastic94,plastic94.jpg,plastic/plastic94.jpg,6,"[[[217, 203, 204], [216, 202, 203], [215, 201,...","(384, 512, 3)"
4800,train,plastic,plastic96,plastic96.jpg,plastic/plastic96.jpg,6,"[[[237, 224, 222], [234, 221, 219], [230, 217,...","(384, 512, 3)"
4801,train,plastic,plastic97,plastic97.jpg,plastic/plastic97.jpg,6,"[[[94, 123, 154], [94, 123, 154], [94, 123, 15...","(384, 512, 3)"


**X, Y** **TEST**

Descargar imágenes para X_test e y_test

In [14]:
path_test = src_path + os.sep + "data" +  os.sep + "test"  +  os.sep + '**'  +  os.sep + '**.jpg'
path_test

'c:\\Users\\Mary\\Desktop\\BootCamp\\Python\\MaryC-MezaR\\data_science_bootcamp_2021\\Machine_Learning_project\\data\\test\\**\\**.jpg'

In [15]:

test_set = make_imag_df(path_test)

In [16]:
test_set = creat_colum (test_set)

In [30]:
test_set

,conjunto,name,id,img,path,label
0,test,battery,battery101,battery101.jpg,battery/battery101.jpg,0
1,test,battery,battery108,battery108.jpg,battery/battery108.jpg,0
2,test,battery,battery122,battery122.jpg,battery/battery122.jpg,0
3,test,battery,battery132,battery132.jpg,battery/battery132.jpg,0
4,test,battery,battery14,battery14.jpg,battery/battery14.jpg,0
...,...,...,...,...,...,...
1198,test,plastic,plastic864,plastic864.jpg,plastic/plastic864.jpg,6
1199,test,plastic,plastic90,plastic90.jpg,plastic/plastic90.jpg,6
1200,test,plastic,plastic93,plastic93.jpg,plastic/plastic93.jpg,6
1201,test,plastic,plastic95,plastic95.jpg,plastic/plastic95.jpg,6


In [17]:
path_path_2 = src_path + os.sep + "data" + os.sep + "test"  +  os.sep
path_path_2

'c:\\Users\\Mary\\Desktop\\BootCamp\\Python\\MaryC-MezaR\\data_science_bootcamp_2021\\Machine_Learning_project\\data\\test\\'

In [18]:
imagenes = []
for i in test_set.path:
    x = path_path_2 + i
    imagenes_rs = cv2.imread(x, 1)
    imagenes.append(cv2.resize(imagenes_rs, (224, 224)))
    
test_set["nombre_img"] = imagenes

In [18]:
test_set

,conjunto,name,id,img,path,label,nombre_img
0,test,battery,battery101,battery101.jpg,battery/battery101.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,test,battery,battery108,battery108.jpg,battery/battery108.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
2,test,battery,battery122,battery122.jpg,battery/battery122.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
3,test,battery,battery132,battery132.jpg,battery/battery132.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
4,test,battery,battery14,battery14.jpg,battery/battery14.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
...,...,...,...,...,...,...,...
1198,test,plastic,plastic864,plastic864.jpg,plastic/plastic864.jpg,6,"[[[182, 191, 200], [185, 194, 203], [184, 192,..."
1199,test,plastic,plastic90,plastic90.jpg,plastic/plastic90.jpg,6,"[[[200, 221, 236], [200, 221, 236], [200, 221,..."
1200,test,plastic,plastic93,plastic93.jpg,plastic/plastic93.jpg,6,"[[[216, 203, 201], [215, 202, 200], [214, 201,..."
1201,test,plastic,plastic95,plastic95.jpg,plastic/plastic95.jpg,6,"[[[255, 242, 240], [255, 242, 240], [255, 242,..."


DEFINIR X & Y

In [19]:
X_train = np.stack(np.array(train_set["nombre_img"]))
y_train = np.array(train_set["label"])
print(X_train.shape)
print(y_train.shape)

(4803, 224, 224, 3)
(4803,)


In [20]:
X_test = np.stack(np.array(test_set["nombre_img"]))
y_test = np.array(test_set["label"])
print(X_test.shape)
print(y_test.shape)

(1203, 224, 224, 3)
(1203,)


RESHAPE

Train

In [21]:


X_train = X_train.reshape(4803, 224, 224, 3)
X_train.shape

(4803, 224, 224, 3)

In [22]:
y_train = y_train.reshape(4803, 1)
y_train.shape

(4803, 1)

In [23]:
# Reserve 10,000 samples for validation
X_val = X_train[-900:]
y_val = y_train[-900:]
X_train = X_train[:-900]
y_train = y_train[:-900]

Test

In [24]:
X_test = X_test.reshape(1203, 224, 224, 3)
X_test.shape

(1203, 224, 224, 3)

In [25]:
y_test = y_test.reshape(1203,1)
y_test.shape

(1203, 1)

In [26]:
datagen = ImageDataGenerator(rescale=1./255)

In [27]:
datagen.fit(X_train)

In [28]:
datagen.fit(X_test)

In [29]:
datagen.fit(X_val)

ENTRENAR EL MODELO BASICO

In [41]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(224, 224 ,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [42]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 64)      256       
_________________________________________________________________
dropout (Dropout)            (None, 112, 112, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 802816)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

COMPILAR EL MODELO

In [44]:

model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [45]:
print("Fit model on training data")
history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=10, verbose=1,
    validation_data=(X_val, y_val),
)

Fit model on training data
Epoch 1/10
61/61 [==============================] - 830s 13s/step - loss: 21.3220 - accuracy: 0.3464 - val_loss: 1354.1324 - val_accuracy: 0.0000e+00
Epoch 2/10
61/61 [==============================] - 805s 13s/step - loss: 1.7769 - accuracy: 0.3059 - val_loss: 251.0861 - val_accuracy: 0.0022
Epoch 3/10
61/61 [==============================] - 543s 9s/step - loss: 1.5202 - accuracy: 0.4028 - val_loss: 53.8479 - val_accuracy: 0.0078
Epoch 4/10
61/61 [==============================] - 618s 10s/step - loss: 1.4309 - accuracy: 0.4522 - val_loss: 77.4831 - val_accuracy: 0.1400
Epoch 5/10
61/61 [==============================] - 687s 11s/step - loss: 1.5066 - accuracy: 0.4543 - val_loss: 40.0102 - val_accuracy: 0.0000e+00
Epoch 6/10
61/61 [==============================] - 620s 10s/step - loss: 1.3970 - accuracy: 0.4491 - val_loss: 774.2455 - val_accuracy: 0.2256
Epoch 7/10
61/61 [==============================] - 628s 10s/step - loss: 1.3089 - accuracy: 0.4924 - v

In [47]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

print('\nTest accuracy:', test_acc, "\nLoss:", test_loss)

38/38 [==============================] - 37s 985ms/step - loss: 2.1864 - accuracy: 0.3791

Test accuracy: 0.37905237078666687 
Loss: 2.186438798904419


In [48]:
model.save("baseline_model.h5")

In [38]:
# volverlo a cargar

model = tf.keras.models.load_model('baseline_model.h5')

## 2do modelo

AUMENTAR REDES 

In [39]:
model_2 = Sequential()
model_2.add(Conv2D(64, (3, 3), input_shape=(224, 224 ,3), padding='same', activation='relu'))
model_2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.2))

model_2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.2))

model_2.add(Flatten())
model_2.add(Dense(256, activation='relu'))
model_2.add(Dense(8, activation='softmax'))

In [40]:


model_2.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [41]:
print("Fit model_2 on training data")
history = model_2.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=20, verbose=1,
    validation_data=(X_val, y_val),
)

Fit model_2 on training data
Epoch 1/20
61/61 [==============================] - 849s 14s/step - loss: 12.7625 - accuracy: 0.5152 - val_loss: 124.3385 - val_accuracy: 0.0111
Epoch 2/20
61/61 [==============================] - 855s 14s/step - loss: 0.8202 - accuracy: 0.7028 - val_loss: 24.7818 - val_accuracy: 0.1700
Epoch 3/20
61/61 [==============================] - 861s 14s/step - loss: 0.5372 - accuracy: 0.8104 - val_loss: 22.2010 - val_accuracy: 0.1289
Epoch 4/20
61/61 [==============================] - 844s 14s/step - loss: 0.3616 - accuracy: 0.8709 - val_loss: 21.6939 - val_accuracy: 0.0878
Epoch 5/20
61/61 [==============================] - 845s 14s/step - loss: 0.2429 - accuracy: 0.9131 - val_loss: 22.0624 - val_accuracy: 0.1667
Epoch 6/20
61/61 [==============================] - 840s 14s/step - loss: 0.1242 - accuracy: 0.9613 - val_loss: 25.9192 - val_accuracy: 0.1044
Epoch 7/20
61/61 [==============================] - 842s 14s/step - loss: 0.0811 - accuracy: 0.9762 - val_loss:

In [42]:
test_loss, test_acc = model_2.evaluate(X_test,  y_test, verbose=1)

print('\nTest accuracy:', test_acc, "\nLoss:", test_loss)

38/38 [==============================] - 44s 1s/step - loss: 15.5935 - accuracy: 0.3026

Test accuracy: 0.3025768995285034 
Loss: 15.593498229980469


In [44]:
model_2.save("model_2.h5")

In [ ]:
# En este modelo, resulta que la exactitud sobre el set de datos es mucho menor que la exactitud sobre el set de entrenamiento. Esta diferencia entre el train y el test se debe a overfitting. Sobre ajuste sucede cuando un modelo de aprendizaje de maquina tiene un rendimiento peor sobre un set de datos nuevo, que nunca antes ha visto comparada con el de entrenamiento. 
# Es por ello que a partir del ultimo modelo, se realizara un data augmentation y así volver a entrenar y mejorar el score 

Guardar segundo modelo

In [ ]:
train_loss, train_acc = model_2.evaluate(X_train,  y_train, verbose=1)

print('\nTest accuracy:', test_acc, "\nLoss:", test_loss)

154/154 [==============================] - 9s 60ms/step - loss: 4.1263 - accuracy: 0.3632

Test accuracy: 0.3631921708583832 
Loss: 4.126265048980713


More Convultion Layers

menos epocas 

In [ ]:
# volverlo a cargar

model_3 = tf.keras.models.load_model('model_2.h5')

In [45]:
model_3 = Sequential()
model_3.add(Conv2D(64, (3, 3), input_shape=(224, 224 ,3), padding='same', activation='relu'))
model_3.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.25))

model_3.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_3.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.25))

model_3.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model_3.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(1024, activation='relu'))
model_3.add(Dense(8, activation='softmax'))

In [46]:
print(model_3.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
dropout_4 (Dropout)          (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 112, 112, 128)    

In [47]:


model_3.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [48]:
print("Fit model_3 on training data")
history = model_3.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=10, verbose=1,
    validation_data=(X_val, y_val),
)

Fit model_2 on training data
Epoch 1/10
61/61 [==============================] - 1157s 19s/step - loss: 14.5630 - accuracy: 0.5045 - val_loss: 970.6505 - val_accuracy: 0.0000e+00
Epoch 2/10
61/61 [==============================] - 1136s 19s/step - loss: 1.0920 - accuracy: 0.6367 - val_loss: 141.3829 - val_accuracy: 0.0056
Epoch 3/10
61/61 [==============================] - 1147s 19s/step - loss: 0.7858 - accuracy: 0.7153 - val_loss: 35.2252 - val_accuracy: 0.0633
Epoch 4/10
61/61 [==============================] - 1177s 19s/step - loss: 0.6733 - accuracy: 0.7504 - val_loss: 16.9968 - val_accuracy: 0.1944
Epoch 5/10
61/61 [==============================] - 1131s 19s/step - loss: 0.5252 - accuracy: 0.8081 - val_loss: 18.7693 - val_accuracy: 0.1767
Epoch 6/10
61/61 [==============================] - 1131s 19s/step - loss: 0.4256 - accuracy: 0.8450 - val_loss: 35.7228 - val_accuracy: 0.0422
Epoch 7/10
61/61 [==============================] - 1129s 19s/step - loss: 0.3678 - accuracy: 0.8665

In [49]:
train_loss, train_acc = model_3.evaluate(X_train,  y_train, verbose=1)

print('\nTest accuracy:', test_acc, "\nLoss:", test_loss)

122/122 [==============================] - 221s 2s/step - loss: 2.3830 - accuracy: 0.7107

Test accuracy: 0.3025768995285034 
Loss: 15.593498229980469


In [50]:
test_loss, test_acc = model_3.evaluate(X_test,  y_test, verbose=1)

print('\nTest accuracy:', test_acc, "\nLoss:", test_loss)

38/38 [==============================] - 68s 2s/step - loss: 9.7165 - accuracy: 0.4106

Test accuracy: 0.41064006090164185 
Loss: 9.716455459594727


Con este accuracy se procede a guardar el modelo como el mejor de esta prueba 

In [51]:
model.save("model_3_best.h5")

Descargar el otro modelo básico 

In [ ]:
files.download("model_3_best.h5")


Data augmentation techniques


There are 5 main techniques for image data augmentation:

Image shifts
Image flips
Image rotations
Image brightness
Image zoom
Since most of the images are well centered and cropped, it doesn't make sense using shifts or zoom, so we are going to keep it to horizontal flips, rotations between -25º and +25º and brightness.


In [33]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator=ImageDataGenerator(rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)



In [30]:
path_model3 = src_path + os.sep + "models" + os.sep 

In [31]:
model_data_augmentation = tf.keras.models.load_model(path_model3 + 'model_3_best.h5')

In [34]:
data_generator.fit(X_train)

In [36]:
model_data_augmentation.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [37]:
print("Fit model_data_augmentation on training data")
history = model_data_augmentation.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=10, verbose=1,
    validation_data=(X_val, y_val),
)

Fit model_data_augmentation on training data
Epoch 1/10
61/61 [==============================] - 638s 10s/step - loss: 1.3550 - accuracy: 0.5165 - val_loss: 164.7543 - val_accuracy: 0.1900
Epoch 2/10
61/61 [==============================] - 607s 10s/step - loss: 1.5895 - accuracy: 0.4158 - val_loss: 556.6712 - val_accuracy: 0.0011
Epoch 3/10
61/61 [==============================] - 595s 10s/step - loss: 1.4463 - accuracy: 0.4497 - val_loss: 77.6210 - val_accuracy: 0.0144
Epoch 4/10
61/61 [==============================] - 625s 10s/step - loss: 1.4867 - accuracy: 0.4415 - val_loss: 23.5270 - val_accuracy: 0.2056
Epoch 5/10
61/61 [==============================] - 658s 11s/step - loss: 1.2266 - accuracy: 0.4876 - val_loss: 10.5378 - val_accuracy: 0.2089
Epoch 6/10
61/61 [==============================] - 608s 10s/step - loss: 1.1992 - accuracy: 0.4955 - val_loss: 3.9146 - val_accuracy: 0.2078
Epoch 7/10
61/61 [==============================] - 570s 9s/step - loss: 1.1146 - accuracy: 0.52

In [38]:
test_loss, test_acc = model_data_augmentation.evaluate(X_test,  y_test, verbose=1)

print('\nTest accuracy:', test_acc, "\nLoss:", test_loss)

38/38 [==============================] - 43s 1s/step - loss: 3.5095 - accuracy: 0.3084

Test accuracy: 0.3083956837654114 
Loss: 3.50949764251709


In [ ]:
#guardar este modelo

In [43]:
path_model = src_path + os.sep + "models" + os.sep 

In [44]:
model_data_augmentation.save(path_model + "model_data_augmentation.h5")

In [ ]:
# se procede a entrenar este modelo con mas redes 

In [ ]:
# rotación de imagenes con una rotación aleatoria para ver si se mejora el accuracy 

In [40]:
data_generator=ImageDataGenerator(brightness_range=[0.2,1.0])


In [41]:
data_generator.fit(X_train)

In [45]:
model_data_augmentation_ran = tf.keras.models.load_model(path_model3 + 'model_data_augmentation.h5')

In [46]:
model_data_augmentation_ran.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [47]:
print("Fit model_data_augmentation on training data")
history = model_data_augmentation_ran.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=10, verbose=1,
    validation_data=(X_val, y_val),
)

Fit model_data_augmentation on training data
Epoch 1/10
61/61 [==============================] - 566s 9s/step - loss: 1.0489 - accuracy: 0.5516 - val_loss: 42.0274 - val_accuracy: 0.1900
Epoch 2/10
61/61 [==============================] - 582s 10s/step - loss: 0.9292 - accuracy: 0.5824 - val_loss: 435.2493 - val_accuracy: 0.2222
Epoch 3/10
61/61 [==============================] - 591s 10s/step - loss: 0.9897 - accuracy: 0.5890 - val_loss: 69.4559 - val_accuracy: 0.1944
Epoch 4/10
61/61 [==============================] - 580s 10s/step - loss: 0.8925 - accuracy: 0.6057 - val_loss: 24.7287 - val_accuracy: 0.0478
Epoch 5/10
61/61 [==============================] - 600s 10s/step - loss: 1.0800 - accuracy: 0.5562 - val_loss: 388.0945 - val_accuracy: 0.0633
Epoch 6/10
61/61 [==============================] - 601s 10s/step - loss: 0.8935 - accuracy: 0.6139 - val_loss: 38.0887 - val_accuracy: 0.0078
Epoch 7/10
61/61 [==============================] - 604s 10s/step - loss: 0.8676 - accuracy: 0.6

In [48]:
model_data_augmentation_ran.save(path_model + "model_data_augmentation_rot_alea.h5")

In [49]:
test_loss, test_acc = model_data_augmentation.evaluate(X_test,  y_test, verbose=1)

print('\nTest accuracy:', test_acc, "\nLoss:", test_loss)

38/38 [==============================] - 37s 959ms/step - loss: 3.5095 - accuracy: 0.3084

Test accuracy: 0.3083956837654114 
Loss: 3.50949764251709


In [ ]:
## Este accuracy ha dado peores resultados que el resto de los modelos entrenados 